This notebook shows how Constant Q-transform from waves can be calculated with usage of nnAudio package (https://github.com/KinWaiCheuk/nnAudio). 

nnAudio is an audio processing toolbox using PyTorch convolutional neural network as its backend.

Have any questions or suggestions? Please comment below.

**<font color='red'>And if you liked this notebook, please upvote it!</font>**

**Changelog**
* v6 - number of processed samples can be now easily changed via num_samples variable
* v5 - added Tukey window
* v4 - added a bandpass filter (idea taken from https://www.kaggle.com/c/g2net-gravitational-wave-detection/discussion/261721#1458564) + wave plots
* v3 - small markup fix :)
* v2 - changed CQT1992v2 to CQT (alias)

## Import packages

In [ ]:
!pip install -q nnAudio

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import signal
import torch
from torch.utils.data import Dataset
from nnAudio.Spectrogram import CQT # CQT is an alias of CQT1992v2

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
num_samples = 4 # first N samples to process

## Define dataset

Let's define a dataset to work with.

In [ ]:
class G2NetDataset(Dataset):
    def __init__(self, paths, targets, use_filter=True): 
        self.paths = paths
        self.targets = targets
        self.use_filter = use_filter
        if self.use_filter:
            self.bHP, self.aHP = signal.butter(8, (20, 500), btype='bandpass', fs=2048)

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):      
        waves = np.load(self.paths[index])
        waves = np.concatenate(waves, axis=0)
        if self.use_filter:
            waves *= signal.tukey(4096*3, 0.2)
            waves = signal.filtfilt(self.bHP, self.aHP, waves)
        waves = waves/np.max(waves)
        targets = self.targets[index]
                
        return {
            "waves": torch.tensor(waves, dtype=torch.float),
            "target": torch.tensor(targets, dtype=torch.long),
        }

## Read training labels

Now we read training labels data, and get npy paths.

In [ ]:
ROOT_DIR = '../input/g2net-gravitational-wave-detection'
df = pd.read_csv(os.path.join(ROOT_DIR, 'training_labels.csv'))
df['path'] = df['id'].apply(lambda x: f'{ROOT_DIR}/train/{x[0]}/{x[1]}/{x[2]}/{x}.npy')

## Demonstrate CQT usage

nnAudio has several implementations of CQT; we will use the recommended one - CQT1992v2 (see https://github.com/KinWaiCheuk/nnAudio/blob/master/Installation/nnAudio/Spectrogram.py for details). You can simply call CQT, since it's an alias of CQT1992v2. If you want other CQT version, you'll have to import it directly.

Let's calculate CQT for 4 first signals with and without usage of a bandpass filter (20-500Hz), and plot results!

In [ ]:
transform = CQT(sr=2048,        # sample rate
                fmin=20,        # min freq
                fmax=1024,      # max freq (set to Nyquist frequency)
                hop_length=64,  # hop length
                verbose=False)  

ds = G2NetDataset(df['path'], df['target'], use_filter=False)
ds_f = G2NetDataset(df['path'], df['target'], use_filter=True)

waves = []
waves_f = []
cqts = []
cqts_f = []
for i in range(num_samples):
    waves.append(ds.__getitem__(i)['waves'])
    waves_f.append(ds_f.__getitem__(i)['waves'])
    cqts.append(transform(waves[i]).squeeze())
    cqts_f.append(transform(waves_f[i]).squeeze())

### Without a filter

In [ ]:
fig, axs = plt.subplots(num_samples)
fig.set_figheight(15)
fig.set_figwidth(15)
for i in range(num_samples):
    nid = df['id'][i]
    ntarget = df['target'][i]
    axs[i].title.set_text(f'{nid}.npy, target: {ntarget}')
    axs[i].plot(waves[i])

In [ ]:
fig, axs = plt.subplots(num_samples)
fig.set_figheight(15)
fig.set_figwidth(15)
for i in range(num_samples):
    nid = df['id'][i]
    ntarget = df['target'][i]
    axs[i].title.set_text(f'{nid}.npy, target: {ntarget}')
    axs[i].pcolormesh(cqts[i])

### With a filter with Tukey window

In [ ]:
fig, axs = plt.subplots(num_samples)
fig.set_figheight(15)
fig.set_figwidth(15)
for i in range(num_samples):
    nid = df['id'][i]
    ntarget = df['target'][i]
    axs[i].title.set_text(f'{nid}.npy, target: {ntarget}')
    axs[i].plot(waves_f[i])

In [ ]:
fig, axs = plt.subplots(num_samples)
fig.set_figheight(15)
fig.set_figwidth(15)
for i in range(num_samples):
    nid = df['id'][i]
    ntarget = df['target'][i]
    axs[i].title.set_text(f'{nid}.npy, target: {ntarget}')
    axs[i].pcolormesh(cqts_f[i])

You can use CQT() as your model block to convert waves to CQT on-the-fly.